In [1]:
import pandas as pd
import os

# Assuming this notebook is in examples/
registry_pth = os.path.join(os.getcwd(), "../evals/registry")

In [2]:
sys_msg = "You will be presented with a DNA sequence and a given letter. The task is to count frequency of given letter in the sequence and count the length of the DNA sequence. After reading the sequence tell me the frequency of given letter and length of the DNA sequence by saying ('X','Y') where 'X' is the frequency and 'Y' is the length of the DNA sequence"
def create_chat_prompt(DNA, letter):
    user_prompt = f"{DNA},{letter}"
    return [
        {"role": "system", "content": sys_msg}, 
        {"role": "user", "content": user_prompt}
    ]

In [3]:
def getIdeal(DNA,letter):
    length = len(DNA)
    count = 0
    for char in DNA:
        if char == letter:
            count += 1
    return f"({count},{length})"

In [4]:
DNA, letter = "AAATTT","A"
question = create_chat_prompt(DNA, letter)
answer = getIdeal(DNA, letter)
print(question,answer)

[{'role': 'system', 'content': "You will be presented with a DNA sequence and a given letter. The task is to count frequency of given letter in the sequence and count the length of the DNA sequence. After reading the sequence tell me the frequency of given letter and length of the DNA sequence by saying ('X','Y') where 'X' is the frequency and 'Y' is the length of the DNA sequence"}, {'role': 'user', 'content': 'AAATTT,A'}] (3,6)


In [5]:
subject_pth = os.path.join(registry_pth, "data", "count_token_freq_dna")
#os.makedirs(subject_pth, exist_ok=True)

In [27]:
def add_to_dataframe( DNA_list, letter_list):
    data_frame = {'input':[],'ideal':[]}
    for DNA, letter in zip(DNA_list,letter_list):
        data_frame['input'].append(create_chat_prompt(DNA,letter))
        data_frame['ideal'].append(getIdeal(DNA,letter))
    return data_frame


In [37]:
resource = pd.read_csv("resource.csv",sep="\t")
seqs = resource['seq']

In [38]:
import random
def truncate_string(string):
    length = random.randint(50,101)
    if len(string) <= length:
        return string
    start = random.randint(0, len(string) - length)
    end = start + length
    return string[start:end]

In [52]:
DNAs = []
letters = []
temp_letter_dict = ['A','T','G','C']
count = 0
N = 20
for seq in seqs:
    if count == N:
        break
    if len(seq)>10:
        DNAs.append(truncate_string(seq))
        letters.append(temp_letter_dict[random.randint(0,3)])
    count += 1
        

In [54]:
data_frame = add_to_dataframe( DNAs, letters)

In [56]:
df = pd.DataFrame(data=data_frame)
samples_pth = os.path.join(subject_pth, "samples.jsonl")
df[["input", "ideal"]].to_json(samples_pth, lines=True, orient="records")

In [61]:
import yaml
eval_id = f"count_token_freq_dna"
registry_yaml = {}
registry_yaml[eval_id] = {
        "id": f"{eval_id}.dev.v0",
        "metrics": ["accuracy"]
    }
registry_yaml[f"{eval_id}.dev.v0"] = {
        "class": "evals.elsuite.basic.match:Match",
        "args": {
            "samples_jsonl": samples_pth,
        }
    }
with open(os.path.join(registry_pth, "evals", "count_token_freq_dna.yaml"), "w") as f:
    yaml.dump(registry_yaml, f)

In [62]:
!oaieval gpt-3.5-turbo count_token_freq_dna

[2023-03-30 19:47:50,436] [registry.py:156] Loading registry from /Users/lizehui/Desktop/workspace/evals/evals/registry/evals
[2023-03-30 19:47:50,491] [registry.py:156] Loading registry from /Users/lizehui/.evals/evals
Traceback (most recent call last):
  File "/Users/lizehui/opt/anaconda3/envs/gpt-eval/bin/oaieval", line 8, in <module>
    sys.exit(main())
  File "/Users/lizehui/Desktop/workspace/evals/evals/cli/oaieval.py", line 248, in main
    run(args, model_resolver=ModelResolver())
  File "/Users/lizehui/Desktop/workspace/evals/evals/cli/oaieval.py", line 157, in run
    assert (
AssertionError: Eval count_token_freq_dna not found. Available: ['aba_mrpc_true_false', 'aba_mrpc_true_false.dev.v0', 'actors-sequence', 'actors-sequence.dev.match-v1', 'anagrams', 'anagrams.test.v1', 'balance-chemical-equation', 'balance-chemical-equation.dev.v0', 'belarusian-lexicon', 'belarusian-lexicon.dev.v0', 'best', 'best.dev.v0', 'bigrams', 'bigrams.dev.v0', 'born-first', 'born-first.dev.v0', '